In [ ]:
%pip install numpy
%pip install pandas
%pip install sklearn
%pip install matplotlib
import numpy as np
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor
from sklearn.impute import KNNImputer
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from scipy.stats import spearmanr
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [20, 15]

In [36]:
df = pd.read_csv("water_potability.csv")
desc = df.describe()
desc

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
count,2785.000000,3276.000000,3276.000000,3276.000000,2495.000000,3276.000000,3276.000000,3114.000000,3276.000000,3276.000000
mean,7.080795,196.369496,22014.092526,7.122277,333.775777,426.205111,14.284970,66.396293,3.966786,0.390110
std,1.594320,32.879761,8768.570828,1.583085,41.416840,80.824064,3.308162,16.175008,0.780382,0.487849
min,0.000000,47.432000,320.942611,0.352000,129.000000,181.483754,2.200000,0.738000,1.450000,0.000000
25%,6.093092,176.850538,15666.690297,6.127421,307.699498,365.734414,12.065801,55.844536,3.439711,0.000000
50%,7.036752,196.967627,20927.833607,7.130299,333.073546,421.884968,14.218338,66.622485,3.955028,0.000000
75%,8.062066,216.667456,27332.762127,8.114887,359.950170,481.792304,16.557652,77.337473,4.500320,1.000000
max,14.000000,323.124000,61227.196008,13.127000,481.030642,753.342620,28.300000,124.000000,6.739000,1.000000


In [ ]:
imputer = KNNImputer(n_neighbors=3)
df_filled = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

In [35]:
def calc_corr(data_frame):
    columns = data_frame.columns
    results = pd.DataFrame(columns=columns, index=columns)
    for column1 in columns:
        for column2 in columns:
            if column1 == column2:
                continue
            corr, _ = spearmanr(data_frame[column1], data_frame[column2])
            results[column1][column2] = corr
    return results
calc_corr(df_filled)


,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
ph,NaN,0.109367,-0.090471,-0.036612,0.027685,0.016529,0.033707,-0.002598,-0.040255,-0.004091
Hardness,0.109367,NaN,-0.052584,-0.024556,-0.096192,-0.032884,0.00334,-0.015155,-0.012855,-0.010606
Solids,-0.090471,-0.052584,NaN,-0.055037,-0.170159,0.021089,0.017881,-0.012979,0.02847,0.026234
Chloramines,-0.036612,-0.024556,-0.055037,NaN,0.032032,-0.016928,-0.01191,0.011601,-0.007909,0.02498
Sulfate,0.027685,-0.096192,-0.170159,0.032032,NaN,-0.018605,0.017764,-0.03053,-0.018213,-0.010207
Conductivity,0.016529,-0.032884,0.021089,-0.016928,-0.018605,NaN,0.021311,-0.002508,0.010342,-0.010386
Organic_carbon,0.033707,0.00334,0.017881,-0.01191,0.017764,0.021311,NaN,-0.00361,-0.02473,-0.026774
Trihalomethanes,-0.002598,-0.015155,-0.012979,0.011601,-0.03053,-0.002508,-0.00361,NaN,-0.030243,0.00657
Turbidity,-0.040255,-0.012855,0.02847,-0.007909,-0.018213,0.010342,-0.02473,-0.030243,NaN,0.001088
Potability,-0.004091,-0.010606,0.026234,0.02498,-0.010207,-0.010386,-0.026774,0.00657,0.001088,NaN


In [51]:
df_filled = df_filled.drop(['ph', 'Conductivity', 'Turbidity', 'Solids'], axis=1)
display(df_filled)

KeyError: "['ph' 'Conductivity' 'Turbidity' 'Solids'] not found in axis"

In [37]:
df_potable = df_filled.query('Potability == 1').drop(columns=['Potability'])
df_unpotable = df_filled.query('Potability == 0').drop(columns=['Potability'])

df_potable = df_potable.reset_index().drop(columns=['index'])
df_unpotable = df_unpotable.reset_index().drop(columns=['index'])

display(df_potable.describe())
display(df_unpotable.describe())

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
count,1278.000000,1278.000000,1278.000000,1278.000000,1278.000000,1278.000000,1278.000000,1278.000000,1278.000000
mean,7.067622,195.800744,22383.991018,7.169338,332.638612,425.383800,14.160893,66.565356,3.968328
std,1.391363,35.547041,9101.010208,1.702988,43.444485,82.048446,3.263907,16.080678,0.780842
min,0.227499,47.432000,728.750830,0.352000,129.000000,201.619737,2.200000,8.175876,1.492207
25%,6.233140,174.330531,15668.985035,6.094134,306.329450,360.939023,12.033897,56.717065,3.430909
50%,7.041030,196.632907,21199.386614,7.215163,333.142134,420.712729,14.162809,66.697224,3.958576
75%,7.887966,218.003420,27973.236446,8.199261,359.655180,484.155911,16.356245,77.204861,4.509569
max,13.175402,323.124000,56488.672413,13.127000,481.030642,695.369528,23.604298,124.000000,6.494249


,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
count,1998.000000,1998.000000,1998.000000,1998.000000,1998.000000,1998.000000,1998.000000,1998.000000,1998.000000
mean,7.085311,196.733292,21777.490788,7.092175,334.092525,426.730454,14.364335,66.304939,3.965800
std,1.589761,31.057540,8543.068788,1.501045,34.053413,80.047317,3.334554,15.769498,0.780282
min,0.000000,98.452931,320.942611,1.683993,203.444521,181.483754,4.371899,0.738000,1.450000
25%,6.095666,177.823265,15663.057382,6.155640,313.004398,368.498530,12.101057,55.995254,3.444062
50%,7.059692,197.123423,20809.618280,7.090334,333.389426,422.229331,14.293508,66.481145,3.948076
75%,8.043015,216.120687,27006.249009,8.066462,354.052560,480.677198,16.649485,77.000519,4.496106
max,14.000000,304.235912,61227.196008,12.653362,460.107069,753.342620,28.300000,120.030077,6.739000


In [38]:
def find_outliers(data_frame):
    clf = LocalOutlierFactor(n_neighbors=3)
    outliers = clf.fit_predict(data_frame)
    outliers_indexes = np.where(outliers == - 1)[0]
    rest_indexes = np.where(outliers == 1)[0]
    outliers = data_frame.loc[outliers_indexes]
    df_without_outliers = data_frame.loc[rest_indexes]
    df_without_outliers = df_without_outliers.reset_index().drop(columns=['index'])
    return outliers, df_without_outliers

In [39]:
outliers_potable, df_pot_clean = find_outliers(df_potable)
outliers_unpotable, df_unpot_clean = find_outliers(df_unpotable)

In [ ]:
display(df_pot_clean.describe())
display(df_unpot_clean.describe())


In [ ]:
display(df_pot_clean.describe())
display(outliers_potable)

In [41]:
display(df_unpot_clean.describe())
display(outliers_unpotable)

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
count,1923.000000,1923.000000,1923.000000,1923.000000,1923.000000,1923.000000,1923.000000,1923.000000,1923.000000
mean,7.068952,197.156957,21758.245459,7.097606,333.889164,426.299014,14.367964,66.369303,3.960486
std,1.587592,30.525192,8383.777978,1.493119,33.168892,77.231055,3.340583,15.737804,0.778338
min,0.000000,98.452931,3413.081633,1.683993,203.444521,210.319182,4.371899,0.738000,1.450000
25%,6.086873,178.544525,15643.924307,6.167605,313.105285,369.241099,12.161396,56.006535,3.440020
50%,7.040245,197.517093,20855.272321,7.086469,333.148318,421.963539,14.298579,66.588849,3.941074
75%,8.035278,216.096228,27000.721100,8.068454,353.374790,479.164849,16.647555,77.029129,4.482006
max,14.000000,304.235912,49456.587108,12.653362,458.441072,672.556999,28.300000,120.030077,6.739000


,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
1,3.716080,129.422921,18630.057858,6.635246,351.285226,592.885359,15.180013,56.329076,4.500656
8,7.607123,118.988579,14285.583854,7.804174,268.646941,389.375566,12.706049,53.928846,3.595017
84,6.395359,133.834229,23827.275015,5.127747,266.908428,573.400295,9.833875,95.026609,4.191218
108,6.098157,189.995846,27776.901052,6.443098,347.930400,245.859632,14.636882,73.424632,3.977362
126,7.216172,252.670469,29368.674184,9.906147,433.952212,465.216621,13.221034,84.198607,5.052737
...,...,...,...,...,...,...,...,...,...
1898,11.496859,186.125296,12253.303389,8.066980,325.409341,260.531758,9.674083,68.073928,5.005193
1902,10.337671,163.276225,21820.295875,7.899521,356.179637,561.206238,14.116324,56.193061,5.063109
1951,6.129386,216.237975,20886.786563,7.326370,303.498672,269.817539,15.252780,104.682553,3.860348
1962,10.817831,141.488391,23989.990151,7.962066,389.392897,361.632055,14.908725,57.164556,3.590841


,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
count,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000
mean,7.504773,185.870518,22270.941024,6.952918,339.306720,437.792566,14.271299,64.654656,4.102046
std,1.598514,41.289592,11989.352912,1.696918,51.875012,133.677570,3.196088,16.586336,0.822331
min,3.344589,107.383327,320.942611,2.458609,225.516628,181.483754,7.715740,25.525267,2.148612
25%,6.516823,155.096401,16433.609785,6.007002,307.673560,307.785516,11.721580,55.617870,3.614389
50%,7.384049,184.869606,20115.733588,7.141372,342.132512,457.551559,14.116324,63.042779,4.102292
75%,8.677412,218.619105,27699.823660,8.015982,363.244189,530.508338,16.490764,73.306054,4.692861
max,11.496859,276.733569,61227.196008,10.338234,460.107069,753.342620,21.977158,108.589414,5.703288


In [ ]:
def clusterize(data_frame, eps):
    normalized_df=(data_frame-data_frame.min())/(data_frame.max()-data_frame.min())
    clustering = DBSCAN(eps=eps).fit(normalized_df)
    output = []
    for label in range(clustering.n_features_in_):
        tmp_indexes = np.where(clustering.labels_ == label)[0]
        tmp = data_frame.loc[tmp_indexes].reset_index().drop(columns=['index'])
        output.append(tmp)
    return output
    

In [ ]:
display(df_pot_clean.describe())
clusters = clusterize(df_pot_clean, 0.2)
for i, cluster in enumerate(clusters):
    if(len(cluster)):
        display(f'Cluster {i}:')
        display(f'Number of elements: {len(cluster)}')
        # print(f'Labels: {len(df_without_outliers)}')
        # print(tmp[['ph', 'Hardness', 'Potability']].describe())
        display(cluster.describe())
        # print(tmp[['ph', 'Hardness', 'Potability']])

for i, col in enumerate(df_pot_clean.columns):
    ax = plt.subplot(3,3,i+1)
    for j, cluster in enumerate(clusters):
        ax.set_title(col)
        ax.plot(cluster[col], [j for _ in range(len(cluster))], '.')
ax

In [ ]:
display(df_pot_clean.describe())
clusters = clusterize(df_unpot_clean, 0.22)
for i, cluster in enumerate(clusters):
    if(len(cluster)):
        display(f'Cluster {i}:')
        display(f'Number of elements: {len(cluster)}')
        # print(f'Labels: {len(df_without_outliers)}')
        # print(tmp[['ph', 'Hardness', 'Potability']].describe())
        display(cluster.describe())
        # print(tmp[['ph', 'Hardness', 'Potability']])
    plt.plot(cluster['Hardness'], [i for _ in range(len(cluster))], '.')
    
for i, col in enumerate(df_pot_clean.columns):
    ax = plt.subplot(3,3,i+1)
    for j, cluster in enumerate(clusters):
        ax.set_title(col)
        ax.plot(cluster[col], [j for _ in range(len(cluster))], '.')
ax

In [ ]:
df_filed_np = df_filled.drop('Potability', axis=1)
df_cov = df_filed_np.cov()
display(df_cov)
e_val, e_vec = np.linalg.eig(df_cov)
e_val_df = pd.DataFrame([e_val], columns=df_cov.columns)
e_val_df_perc = 100 * e_val_df / sum(e_val)
display(e_val_df_perc)